In [12]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow import keras
from keras import layers

In [2]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\rnmpr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/autompg-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\rnmpr\.cache\kagglehub\datasets\uciml\autompg-dataset\versions\3


In [14]:
df = pd.read_csv("data/auto-mpg.csv") 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mpg            398 non-null    float64
 1    cylinders     398 non-null    int64  
 2    displacement  398 non-null    float64
 3    horsepower    398 non-null    object 
 4    weight        398 non-null    int64  
 5    acceleration  398 non-null    float64
 6    model year    398 non-null    int64  
 7    origin        398 non-null    int64  
 8    car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [27]:
df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace from column names
df.drop('displacement', axis=1, inplace=True)  # Drop the 'displacement' column 
df.columns

Index(['mpg', 'cylinders', 'horsepower', 'weight', 'acceleration',
       'model year', 'origin', 'car name'],
      dtype='object')

In [16]:
df['horsepower'].unique()  # Check unique values in 'horsepower' column

array([' 130       ', ' 165       ', ' 150       ', ' 140       ',
       ' 198       ', ' 220       ', ' 215       ', ' 225       ',
       ' 190       ', ' 170       ', ' 160       ', ' 95        ',
       ' 97        ', ' 85        ', ' 88        ', ' 46        ',
       ' 87        ', ' 90        ', ' 113       ', ' 200       ',
       ' 210       ', ' 193       ', ' ?         ', ' 100       ',
       ' 105       ', ' 175       ', ' 153       ', ' 180       ',
       ' 110       ', ' 72        ', ' 86        ', ' 70        ',
       ' 76        ', ' 65        ', ' 69        ', ' 60        ',
       ' 80        ', ' 54        ', ' 208       ', ' 155       ',
       ' 112       ', ' 92        ', ' 145       ', ' 137       ',
       ' 158       ', ' 167       ', ' 94        ', ' 107       ',
       ' 230       ', ' 49        ', ' 75        ', ' 91        ',
       ' 122       ', ' 67        ', ' 83        ', ' 78        ',
       ' 52        ', ' 61        ', ' 93        ', ' 148     

In [28]:
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce') # Convert 'horsepower' to numeric, coercing errors to NaN
df.dropna(subset=['horsepower'], inplace=True) # Drop rows where 'horsepower' is NaN

In [29]:
df['horsepower'] = df['horsepower'].astype(int)  # Convert 'horsepower' to float

In [30]:
features = df.drop(['mpg','car name'], axis=1)  # Features: all columns except 'mpg' and 'car name'
target = df['mpg'].values  # Target variable: 'mpg'

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42) # Split the dataset into training and test sets0
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (313, 6) (313,)
Test set shape: (79, 6) (79,)


In [31]:
features.head()  # Display the first few rows of the features DataFrame

,cylinders,horsepower,weight,acceleration,model year,origin
0,8,130,3504,12.0,70,1
1,8,165,3693,11.5,70,1
2,8,150,3436,11.0,70,1
3,8,150,3433,12.0,70,1
4,8,140,3449,10.5,70,1


In [32]:
AUTO = tf.data.experimental.AUTOTUNE # Create a TensorFlow dataset from the training data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train)).batch(32).prefetch(AUTO) 

test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test)).batch(32).prefetch(AUTO) # Create a TensorFlow dataset from the test data

In [33]:
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[6]),
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(1, activation='relu')
])

C:\Users\rnmpr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.compile(optimizer='adam', loss='mse', metrics=['mape'])  # Compile the model with Adam optimizer, mean squared error loss, and mean absolute error metric

In [35]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 256)            │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,889 (273.00 KB)

 Trainable params: 68,865 (269.00 KB)

 Non-trainable params: 1,024 (4.00 KB)

In [36]:
history = model.fit(train_dataset,
                    epochs=50,
                    validation_data=test_dataset)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 503.1601 - mape: 93.2535 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 376.9608 - mape: 84.1763 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 324.8589 - mape: 79.2532 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 285.1885 - mape: 74.9839 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 256.8807 - mape: 71.4842 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 227.6147 - mape: 67.2202 - val_loss: 572.6137 - val_mape: 100.0000
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 206.8005 - mape: 63.7510 - val_loss: 571.0467 - val_mape: 99.9384
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 187.5244 - mape: 60.4660 - val_loss: 558.4521 - val_mape: 99.3080
E